In [207]:
import pandas as pd
from carball.generated.api import game_pb2
from carball.generated.api.stats import events_pb2
from carball.generated.api.metadata import game_metadata_pb2
from typing import *
from carball.analysis.utils import pandas_manager
import seaborn as sns

In [32]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


/home/march/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['Text', 'f', 'cast']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [33]:
with open('CC88B14F4E967268FEB550902A22C1A6.pts', 'rb') as f:
    game = game_pb2.Game()
    game.ParseFromString(f.read())

In [202]:
goals = game.game_metadata.goals

In [203]:
all_hits = game.game_stats.hits

In [204]:
sorted_hits = sorted(all_hits, key=lambda h: h.frame_number)

In [208]:
def hits_before_frame(goals: List[game_metadata_pb2.Goal], hits: List[events_pb2.Hit]) -> List[events_pb2.Hit]:
    it = iter(hits)
    last_h = next(it)
    for g in goals:
        try:
            next_h = next(it)
            while next_h.frame_number < g:
                last_h = next_h
                next_h = next(it)
        except StopIteration:
            yield last_h
            return
        yield last_h

In [209]:
prev_hits = list(hits_before_frame(goal_frames, sorted_hits))

In [41]:
with open('CC88B14F4E967268FEB550902A22C1A6.gzip', 'rb') as f:
    df = pandas_manager.PandasManager.read_numpy_from_memory(f)

In [197]:
ball_trayectories = [
    [(r[1]['pos_x'], r[1]['pos_y'], r[1]['pos_z']) for r in df.ball[hit.frame_number:goal_frame].iterrows()]
    for hit, goal_frame in zip(prev_hits, goal_frames)
]

In [210]:
goal_color = [[p.is_orange for p in game.players if p.id == g.player_id][0]
              for g in goals]

In [43]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for bt in ball_trayectories:
    xs, ys, zs = zip(*bt)
    ax.plot(xs, ys, zs)
ax.set_xlabel('x')
ax.set_ylabel('y')

Text(0.5,0,'y')

In [44]:
pos_columns = ['pos_x', 'pos_y', 'pos_z']

In [45]:
actors_pos = pd.concat([
    pd.Series(df.xs(pos_c, axis=1, level=1).melt().value, name=pos_c) 
    for pos_c in pos_columns
], axis=1)

In [46]:
actors = [n for n in df.columns.levels[0] if n != 'game']

In [47]:
actors_data = df[actors].reset_index(0).melt(id_vars='index', var_name=['actor', 'coord'])

In [82]:
actors_pos_round = pd.concat([
    actors_pos['pos_y'].apply(lambda x: np.round(x)),
    actors_pos['pos_x']
], axis=1)

In [83]:
ys_max = actors_pos_round.groupby('pos_y').agg(max)
ys_min = actors_pos_round.groupby('pos_y').agg(min)

In [85]:
scatter(ys_max.values, ys_max.index, marker='x', s=1)
scatter(ys_min.values, ys_min.index, marker='x', s=1)

In [78]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for bt in ball_trayectories:
    ax.scatter(actors_pos['pos_x'], actors_pos['pos_y'], actors_pos['pos_z'])
ax.set_xlabel('x')
ax.set_ylabel('y')

Text(0.5,0,'y')

In [86]:
from scipy.spatial import ConvexHull

In [125]:
points = actors_pos.dropna().values

In [126]:
hull = ConvexHull(points)

In [219]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
vertices = points[hull.vertices]
vertices_f = vertices[vertices[:, 2] < 1000]
vertices_f[:, 2] = [min(n, 300) for n in vertices_f[:, 2]]
vertices_left = vertices_f[vertices_f[:, 1] < 300]
vertices_right = vertices_f[vertices_f[:, 1] > -300]
ax.plot_trisurf(*zip(*vertices_left), color='orange', alpha=.3)
ax.plot_trisurf(*zip(*vertices_right), color='blue', alpha=.3)
ax.set_xlabel('x')
ax.set_ylabel('y')
for idx, (bt, col) in enumerate(zip(ball_trayectories, goal_color)):
    ax.plot3D(*zip(*bt))